In [233]:
%reset

In [1]:
# we are trying to predict the all nba team selections for the 2024 season
import pandas as pd

In [73]:
stats = pd.read_csv("player_all_defense_stats.csv")
stats_2024 = pd.read_csv("player_2024_stats.csv")

In [11]:
stats

,Unnamed: 0,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,Pts Max,Share,Team,W,L,W/L%,GB,PS/G,PA/G,SRS
0,0,Alan Henderson,C,30.0,ATL,82.0,3.0,18.2,1.9,4.0,...,0.0,0.0,Atlanta Hawks,35.0,47.0,0.427,15.0,94.1,97.6,-3.87
1,1,Amal McCaskill,C,29.0,ATL,11.0,0.0,6.4,0.4,1.5,...,0.0,0.0,Atlanta Hawks,35.0,47.0,0.427,15.0,94.1,97.6,-3.87
2,2,Antonio Harvey,PF,32.0,ATL,4.0,0.0,8.0,0.5,1.3,...,0.0,0.0,Atlanta Hawks,35.0,47.0,0.427,15.0,94.1,97.6,-3.87
3,3,Brandon Williams,SF,27.0,ATL,6.0,0.0,3.2,0.2,1.2,...,0.0,0.0,Atlanta Hawks,35.0,47.0,0.427,15.0,94.1,97.6,-3.87
4,4,Chris Crawford,PF,27.0,ATL,5.0,0.0,7.6,1.6,2.6,...,0.0,0.0,Atlanta Hawks,35.0,47.0,0.427,15.0,94.1,97.6,-3.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10194,10194,Monte Morris,PG,27.0,WAS,62.0,61.0,27.3,4.0,8.3,...,0.0,0.0,Washington Wizards,35.0,47.0,0.427,9.0,113.2,114.4,-1.06
10195,10195,Quenton Jackson,PG,24.0,WAS,9.0,0.0,15.0,2.1,4.7,...,0.0,0.0,Washington Wizards,35.0,47.0,0.427,9.0,113.2,114.4,-1.06
10196,10196,Taj Gibson,C,37.0,WAS,49.0,2.0,9.8,1.3,2.6,...,0.0,0.0,Washington Wizards,35.0,47.0,0.427,9.0,113.2,114.4,-1.06
10197,10197,Vernon Carey Jr.,C,21.0,WAS,11.0,0.0,2.5,0.2,0.7,...,0.0,0.0,Washington Wizards,35.0,47.0,0.427,9.0,113.2,114.4,-1.06


In [74]:
del stats["Unnamed: 0"] #unneeded column
del stats_2024["Unnamed: 0"]

In [75]:
stats = stats.fillna(0) # i am making it so that all values that are na, are being filled in with 0
stats_2024 = stats_2024.fillna(0)

In [76]:
# There is a new rule in which a player must play at least 65 games in order to be eligible for an all nba team recognition. https://www.cbssports.com/nba/news/tyrese-haliburton-among-players-in-danger-of-missing-all-nba-teams-awards-because-of-leagues-65-game-rule/#:~:text=Perhaps%20the%20most%20notable%20aspect,MVP
stats["Eligibility_65G"] = (stats["G"] >= 65).astype(int)
stats_2024["Eligibility_65G"] = (stats_2024["G"] >= 65).astype(int)

In [15]:
predictors =  ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
        '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'W', 'L', 'W/L%', 'GB', 'PS/G','PA/G', 'SRS', 'Eligibility_65G']

In [16]:
# We are now going to split the data into training and test data
train = stats[stats["Year"] < 2023]
test = stats[stats["Year"] == 2023]

In [77]:
stats2 = stats[:]
stat_ratios = stats2[["PTS", "AST", "STL", "BLK", "3P", "Year"]].groupby("Year").apply(lambda x: x/x.mean())
stat_ratios.reset_index(drop=True, inplace=True)
stats2.reset_index(drop=True, inplace=True)
stats2[["PTS_R", "AST_R", "STL_R", "BLK_R", "3P_R"]] = stat_ratios[["PTS", "AST", "STL", "BLK", "3P"]]
predictors2 = predictors + ["PTS_R", "AST_R", "STL_R", "BLK_R", "3P_R"]

/var/folders/7y/x2g2fgds6c9d9xrjrvscg14r0000gn/T/ipykernel_72445/83561794.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stat_ratios = stats2[["PTS", "AST", "STL", "BLK", "3P", "Year"]].groupby("Year").apply(lambda x: x/x.mean())
/var/folders/7y/x2g2fgds6c9d9xrjrvscg14r0000gn/T/ipykernel_72445/83561794.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats2[["PTS_R", "AST_R", "STL_R", "BLK_R", "3P_R"]] = stat_ratios[["PTS", "AST", "STL", "BLK", "3P"

In [78]:
stats_2024_2 = stats_2024[:]
stat_ratios_2024 = stats_2024_2[["PTS", "AST", "STL", "BLK", "3P", "Year"]].groupby("Year").apply(lambda x: x/x.mean())
stat_ratios_2024.reset_index(drop=True, inplace=True)
stats_2024_2.reset_index(drop=True, inplace=True)
stats_2024_2[["PTS_R", "AST_R", "STL_R", "BLK_R", "3P_R"]] = stat_ratios_2024[["PTS", "AST", "STL", "BLK", "3P"]]

/var/folders/7y/x2g2fgds6c9d9xrjrvscg14r0000gn/T/ipykernel_72445/1787446280.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stat_ratios_2024 = stats_2024_2[["PTS", "AST", "STL", "BLK", "3P", "Year"]].groupby("Year").apply(lambda x: x/x.mean())
/var/folders/7y/x2g2fgds6c9d9xrjrvscg14r0000gn/T/ipykernel_72445/1787446280.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats_2024_2[["PTS_R", "AST_R", "STL_R", "BLK_R", "3P_R"]] = stat_ratios_2024[["PTS",

In [19]:
train2 = stats2[stats2["Year"] < 2023]
test2 = stats2[stats2["Year"] == 2023]

In [20]:
from sklearn.linear_model import Ridge

reg = Ridge(alpha= 1.0)
reg2 = Ridge(alpha=1.0)

In [21]:
reg.fit(train[predictors], train["Share"])
reg2.fit(train2[predictors2], train2["Share"])

Ridge()

In [22]:
# Using Average precision for error metric (if I predicted correctly from the top 15, then I get a 1, and if i didn't it will give me a score based on how far off the rpediction was)
def find_ap(combination):
    actual = combination.sort_values("Share", ascending=False).head(5)
    predicted = combination.sort_values("predictions", ascending=False)
    ps = []
    found = 0
    seen = 1
    for index, row in predicted.iterrows():
        if row["Player"] in actual["Player"].values:
            found += 1
            ps.append(found/seen)
        seen +=1
    return sum(ps) / len(ps)

In [23]:
def add_ranks(predictions):
    predictions = predictions.sort_values("predictions", ascending=False)
    predictions["Predicted_Rk"] = list(range(1,predictions.shape[0]+1))
    predictions = predictions.sort_values("Share", ascending=False)
    predictions["Rk"] = list(range(1,predictions.shape[0]+1))
    predictions["Diff"] = (predictions["Rk"] - predictions["Predicted_Rk"])
    return predictions

In [24]:
# this backtest function just makes it so that we have a more concrete way of testing the data, it includes the use of the other functions that we wrote that makes it easier to easity use this 
from sklearn.metrics import r2_score
def backtest(stats, model, years, predictors):
    aps = []
    r2_scores = []
    all_predictions = []

    for year in years:
        train = stats[stats["Year"] < year]
        test = stats[stats["Year"] == year]

        model.fit(train[predictors], train["Share"])
        predictions = model.predict(test[predictors])
        predictions_df = pd.DataFrame(predictions, columns=["predictions"], index=test.index)

        combination = pd.concat([test[["Player", "Share"]], predictions_df], axis=1)
        combination = add_ranks(combination)
        all_predictions.append(combination)

        ap = find_ap(combination)
        aps.append(ap)

        r2 = r2_score(test["Share"], predictions)
        r2_scores.append(r2)

    mean_ap = sum(aps) / len(aps)
    mean_r2 = sum(r2_scores) / len(r2_scores)

    return mean_ap, mean_r2, aps, pd.concat(all_predictions)


In [25]:
years = list(range(2003, 2024))

In [26]:
Model_error_MAP = {}
Model_error_r2 = {}
Model_error_MAP_regular_model = {}
Model_error_r2_regular_model = {}

In [27]:
from sklearn.linear_model import Ridge

# Ridge Regression 
alpha_values = [0.001, 0.01, 0.1, 1, 10]
results1 = {}
results2 = {}
for alpha in alpha_values:
    reg = Ridge(alpha=alpha, random_state=10)
    mean_ap, mean_r2, aps, all_predictions = backtest(stats, reg, years[5:], predictors)
    results1[alpha] = mean_ap
    results2[alpha] = mean_r2

best_alpha1 = max(results1, key=results1.get)
best_alpha2 = max(results2, key=results2.get)

reg1_regular = Ridge(alpha=best_alpha1, random_state=10)
mean_ap_1, mean_r2_1, aps, all_predictions = backtest(stats, reg1_regular, years[5:], predictors)

reg2_regular = Ridge(alpha=best_alpha2, random_state=10)
mean_ap_2, mean_r2_2, aps, all_predictions = backtest(stats, reg2_regular, years[5:], predictors)

Model_error_MAP_regular_model["Ridge"] = mean_ap_1
Model_error_r2_regular_model["Ridge"] = mean_r2_2


In [28]:
# RIDGE REGRESSION WITH FEATURED ENGINEERED VARIABLES
alpha_values = [0.001, 0.01, 0.1, 1, 10]
results1 = {}
results2 = {}
for alpha in alpha_values:
    reg = Ridge(alpha=alpha, random_state=10)
    mean_ap, mean_r2, aps, all_predictions = backtest(stats2, reg, years[5:], predictors2)
    results1[alpha] = mean_ap
    results2[alpha] = mean_r2

best_alpha1 = max(results1, key=results1.get)
best_alpha2 = max(results2, key=results2.get)

reg1 = Ridge(alpha=best_alpha1, random_state=10)
mean_ap_1, mean_r2_1, aps, all_predictions = backtest(stats2, reg1, years[5:], predictors2)

reg2 = Ridge(alpha=best_alpha2, random_state=10)
mean_ap_2, mean_r2_2, aps, all_predictions = backtest(stats2, reg2, years[5:], predictors2)

Model_error_MAP["Ridge"] = mean_ap_1
Model_error_r2["Ridge"] = mean_r2_2

In [29]:
# prediction using ridge repgression with alpha for best MAP!
over_65_games_2024 = stats_2024[stats_2024["G"] >= 65]
predictions_2024 = reg1_regular.predict(over_65_games_2024[predictors])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024.index)
combination_2024 = pd.concat([over_65_games_2024[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
314,Giannis Antetokounmpo,0.231150
398,Shai Gilgeous-Alexander,0.197557
242,Anthony Davis,0.195075
511,Victor Wembanyama,0.192290
143,Nikola Jokić,0.183903
340,Rudy Gobert,0.167068
124,Luka Dončić,0.166046
385,Chet Holmgren,0.148823
50,Nic Claxton,0.128957
254,LeBron James,0.128950


In [30]:
# prediction using ridge repgression with alpha for best r2 score!
over_65_games_2024 = stats_2024[stats_2024["G"] >= 65]
predictions_2024 = reg2_regular.predict(over_65_games_2024[predictors])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024.index)
combination_2024 = pd.concat([over_65_games_2024[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
314,Giannis Antetokounmpo,0.231150
398,Shai Gilgeous-Alexander,0.197557
242,Anthony Davis,0.195075
511,Victor Wembanyama,0.192290
143,Nikola Jokić,0.183903
340,Rudy Gobert,0.167068
124,Luka Dončić,0.166046
385,Chet Holmgren,0.148823
50,Nic Claxton,0.128957
254,LeBron James,0.128950


In [31]:
# prediction using ridge repgression WITH FEATURED VARIABLES for best alpha value for map score!
over_65_games_2024 = stats_2024_2[stats_2024_2["G"] >= 65]
predictions_2024 = reg1.predict(over_65_games_2024[predictors2])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024.index)
combination_2024 = pd.concat([over_65_games_2024[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
314,Giannis Antetokounmpo,0.230629
398,Shai Gilgeous-Alexander,0.199698
242,Anthony Davis,0.189479
143,Nikola Jokić,0.184388
511,Victor Wembanyama,0.180894
124,Luka Dončić,0.166971
340,Rudy Gobert,0.160789
385,Chet Holmgren,0.141123
254,LeBron James,0.130328
223,Tyrese Haliburton,0.128062


In [32]:
# prediction using ridge repgression WITH FEATURED VARIABLES for best alpha value for r2 score!
over_65_games_2024 = stats_2024_2[stats_2024_2["G"] >= 65]
predictions_2024 = reg2.predict(over_65_games_2024[predictors2])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024.index)
combination_2024 = pd.concat([over_65_games_2024[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
314,Giannis Antetokounmpo,0.230629
398,Shai Gilgeous-Alexander,0.199698
242,Anthony Davis,0.189479
143,Nikola Jokić,0.184388
511,Victor Wembanyama,0.180894
124,Luka Dončić,0.166971
340,Rudy Gobert,0.160789
385,Chet Holmgren,0.141123
254,LeBron James,0.130328
223,Tyrese Haliburton,0.128062


In [33]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=50, random_state=1, min_samples_split=5)
mean_ap, mean_r2, aps, all_predictions = backtest(stats, rf, years[5:], predictors)
Model_error_MAP_regular_model["Random Forest"] = mean_ap
Model_error_r2_regular_model["Random Forest"] = mean_r2

In [34]:
rf2 = RandomForestRegressor(n_estimators=50, random_state=1, min_samples_split=5)
mean_ap, mean_r2, aps, all_predictions = backtest(stats2, rf2, years[5:], predictors2)
Model_error_MAP["Random Forest"] = mean_ap
Model_error_r2["Random Forest"] = mean_r2

In [35]:
#Prediction using RandomForest Regressor:
over_65_games_2024 = stats_2024[stats_2024["G"] >= 65]
predictions_2024 = rf.predict(over_65_games_2024[predictors])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024.index)
combination_2024 = pd.concat([over_65_games_2024[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
398,Shai Gilgeous-Alexander,0.582272
385,Chet Holmgren,0.392814
143,Nikola Jokić,0.379356
314,Giannis Antetokounmpo,0.372399
340,Rudy Gobert,0.349168
242,Anthony Davis,0.287083
232,Kawhi Leonard,0.279958
511,Victor Wembanyama,0.263559
480,Domantas Sabonis,0.259874
238,Paul George,0.246309


In [36]:
#Prediction using RandomForest Regressor with engineered features:
over_65_games_2024_2 = stats_2024_2[stats_2024_2["G"] >= 65]
predictions_2024 = rf2.predict(over_65_games_2024_2[predictors2])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024_2.index)
combination_2024 = pd.concat([over_65_games_2024_2[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
398,Shai Gilgeous-Alexander,0.527111
385,Chet Holmgren,0.417062
314,Giannis Antetokounmpo,0.402850
143,Nikola Jokić,0.384531
232,Kawhi Leonard,0.384410
340,Rudy Gobert,0.345930
511,Victor Wembanyama,0.319154
124,Luka Dončić,0.276808
480,Domantas Sabonis,0.273880
242,Anthony Davis,0.269427


In [37]:
import xgboost as xgb
xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, use_label_encoder=False, eval_metric='map', random_state=1)
mean_ap, mean_r2, aps, all_predictions = backtest(stats, xgb_model, years[5:], predictors)
Model_error_MAP_regular_model["XGBoost"] = mean_ap
Model_error_r2_regular_model["XGBoost"] = mean_r2

In [38]:
xgb_model2 = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, use_label_encoder=False, eval_metric='map', random_state=1)
mean_ap, mean_r2, aps, all_predictions = backtest(stats2, xgb_model2, years[5:], predictors2)
Model_error_MAP["XGBoost"] = mean_ap
Model_error_r2["XGBoost"] = mean_r2

In [39]:
#Prediction using XGBoost Regressor
over_65_games_2024 = stats_2024[stats_2024["G"] >= 65]
predictions_2024 = xgb_model.predict(over_65_games_2024[predictors])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024.index)
combination_2024 = pd.concat([over_65_games_2024[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
398,Shai Gilgeous-Alexander,0.490210
314,Giannis Antetokounmpo,0.394323
385,Chet Holmgren,0.389972
340,Rudy Gobert,0.354114
143,Nikola Jokić,0.291654
511,Victor Wembanyama,0.271863
242,Anthony Davis,0.252837
19,Derrick White,0.175332
24,Jayson Tatum,0.161130
23,Jaylen Brown,0.158673


In [40]:
#Prediction using XGBoost Regressor with engineered variables:
over_65_games_2024_2 = stats_2024_2[stats_2024_2["G"] >= 65]
predictions_2024 = xgb_model2.predict(over_65_games_2024_2[predictors2])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024_2.index)
combination_2024 = pd.concat([over_65_games_2024_2[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
398,Shai Gilgeous-Alexander,0.455099
314,Giannis Antetokounmpo,0.347342
340,Rudy Gobert,0.345731
143,Nikola Jokić,0.326073
385,Chet Holmgren,0.303308
242,Anthony Davis,0.271440
511,Victor Wembanyama,0.265279
24,Jayson Tatum,0.171772
310,Brook Lopez,0.148822
19,Derrick White,0.148270


In [83]:
stats_2024_2[stats_2024_2["Player"] == "LeBron James"].columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'Team',
       'W', 'L', 'W/L%', 'GB', 'PS/G', 'PA/G', 'SRS', 'Eligibility_65G',
       'PTS_R', 'AST_R', 'STL_R', 'BLK_R', '3P_R'],
      dtype='object')

In [80]:
stats_2024_2

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,GB,PS/G,PA/G,SRS,Eligibility_65G,PTS_R,AST_R,STL_R,BLK_R,3P_R
0,AJ Griffin,SF,20,ATL,20,0,8.6,0.9,3.1,0.290,...,11.0,118.3,120.5,-2.38,0,0.284926,0.149895,0.169131,0.248264,0.531401
1,Bogdan Bogdanović,SG,31,ATL,79,33,30.4,6.0,13.9,0.428,...,11.0,118.3,120.5,-2.38,1,2.006351,1.548917,2.029568,0.744792,3.188406
2,Bruno Fernando,C,25,ATL,45,2,15.2,2.4,4.2,0.583,...,11.0,118.3,120.5,-2.38,0,0.747930,0.499651,1.014784,1.489583,0.000000
3,Clint Capela,C,29,ATL,73,73,25.8,4.8,8.5,0.571,...,11.0,118.3,120.5,-2.38,1,1.365268,0.599581,1.014784,3.723958,0.000000
4,De'Andre Hunter,SF,26,ATL,57,37,29.5,5.3,11.6,0.459,...,11.0,118.3,120.5,-2.38,0,1.852016,0.749476,1.183915,0.744792,2.231884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,Marvin Bagley III,C,24,WAS,50,25,21.1,4.8,8.2,0.586,...,32.0,113.7,123.0,-9.29,0,1.389012,0.549616,0.676523,1.737847,0.212560
568,Patrick Baldwin Jr.,SF,21,WAS,38,7,13.0,1.6,4.1,0.381,...,32.0,113.7,123.0,-9.29,0,0.522364,0.399720,0.845653,0.993056,0.850242
569,Richaun Holmes,C-PF,30,WAS,40,10,13.9,2.1,3.7,0.558,...,32.0,113.7,123.0,-9.29,0,0.593595,0.299790,0.507392,1.241319,0.000000
570,Tristan Vukcevic,C,20,WAS,10,4,15.3,2.9,6.7,0.433,...,32.0,113.7,123.0,-9.29,0,1.009111,0.649546,0.845653,1.737847,1.062802


In [45]:
# Lasso REGRESSION
from sklearn.linear_model import Lasso
 
alpha_values = [0.001, 0.01, 0.1, 1, 10]
results1 = {}
results2 = {}
for alpha in alpha_values:
    reg = Lasso(alpha=alpha, random_state=10)
    mean_ap, mean_r2, aps, all_predictions = backtest(stats, reg, years[5:], predictors)
    results1[alpha] = mean_ap
    results2[alpha] = mean_r2

best_alpha1 = max(results1, key=results1.get)
best_alpha2 = max(results2, key=results2.get)

leg1_regular = Lasso(alpha=best_alpha1, random_state=10)
mean_ap_1, mean_r2_1, aps, all_predictions = backtest(stats, leg1_regular, years[5:], predictors)

leg2_regular = Lasso(alpha=best_alpha2, random_state=10)
mean_ap_2, mean_r2_2, aps, all_predictions = backtest(stats, leg2_regular, years[5:], predictors)

Model_error_MAP_regular_model["Lasso"] = mean_ap_1
Model_error_r2_regular_model["Lasso"] = mean_r2_2

In [46]:
from sklearn.linear_model import Lasso
 # RIDGE REGRESSION WITH FEATURED ENGINEERED VARIABLES
alpha_values = [0.001, 0.01, 0.1, 1, 10]
results1 = {}
results2 = {}
for alpha in alpha_values:
    reg = Lasso(alpha=alpha, random_state=10)
    mean_ap, mean_r2, aps, all_predictions = backtest(stats, reg, years[5:], predictors)
    results1[alpha] = mean_ap
    results2[alpha] = mean_r2

best_alpha1 = max(results1, key=results1.get)
best_alpha2 = max(results2, key=results2.get)

leg1 = Lasso(alpha=best_alpha1, random_state=10)
mean_ap_1, mean_r2_1, aps, all_predictions = backtest(stats2, leg1, years[5:], predictors2)

leg2 = Lasso(alpha=best_alpha2, random_state=10)
mean_ap_2, mean_r2_2, aps, all_predictions = backtest(stats2, leg2, years[5:], predictors2)

Model_error_MAP["Lasso"] = mean_ap_1
Model_error_r2["Lasso"] = mean_r2_2

In [47]:
#Prediction using Lasso Regression for alpha with best MAP:
over_65_games_2024 = stats_2024[stats_2024["G"] >= 65]
predictions_2024 = leg1_regular.predict(over_65_games_2024[predictors])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024.index)
combination_2024 = pd.concat([over_65_games_2024[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
511,Victor Wembanyama,0.177045
242,Anthony Davis,0.163416
398,Shai Gilgeous-Alexander,0.155919
314,Giannis Antetokounmpo,0.151326
340,Rudy Gobert,0.149871
143,Nikola Jokić,0.147721
124,Luka Dončić,0.140432
385,Chet Holmgren,0.126657
229,James Harden,0.126067
24,Jayson Tatum,0.116264


In [48]:
#Prediction using Lasso Regression for alpha with best r2 score:
over_65_games_2024 = stats_2024[stats_2024["G"] >= 65]
predictions_2024 = leg2_regular.predict(over_65_games_2024[predictors])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024.index)
combination_2024 = pd.concat([over_65_games_2024[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
511,Victor Wembanyama,0.177045
242,Anthony Davis,0.163416
398,Shai Gilgeous-Alexander,0.155919
314,Giannis Antetokounmpo,0.151326
340,Rudy Gobert,0.149871
143,Nikola Jokić,0.147721
124,Luka Dončić,0.140432
385,Chet Holmgren,0.126657
229,James Harden,0.126067
24,Jayson Tatum,0.116264


In [49]:
#Prediction using Lasso with engineered variables with best MAP:
over_65_games_2024_2 = stats_2024_2[stats_2024_2["G"] >= 65]
predictions_2024 = leg1.predict(over_65_games_2024_2[predictors2])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024_2.index)
combination_2024 = pd.concat([over_65_games_2024_2[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
511,Victor Wembanyama,0.173260
242,Anthony Davis,0.161124
398,Shai Gilgeous-Alexander,0.155378
314,Giannis Antetokounmpo,0.150427
340,Rudy Gobert,0.147787
143,Nikola Jokić,0.147054
124,Luka Dončić,0.140223
229,James Harden,0.125578
385,Chet Holmgren,0.124433
24,Jayson Tatum,0.115960


In [50]:
#Prediction using Lasso with engineered variables with best r2:
over_65_games_2024_2 = stats_2024_2[stats_2024_2["G"] >= 65]
predictions_2024 = leg2.predict(over_65_games_2024_2[predictors2])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024_2.index)
combination_2024 = pd.concat([over_65_games_2024_2[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
511,Victor Wembanyama,0.173260
242,Anthony Davis,0.161124
398,Shai Gilgeous-Alexander,0.155378
314,Giannis Antetokounmpo,0.150427
340,Rudy Gobert,0.147787
143,Nikola Jokić,0.147054
124,Luka Dončić,0.140223
229,James Harden,0.125578
385,Chet Holmgren,0.124433
24,Jayson Tatum,0.115960


In [51]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
mean_ap, mean_r2, aps, all_predictions = backtest(stats, linear_model, years[5:], predictors)
Model_error_MAP_regular_model["Linear"] = mean_ap_1
Model_error_r2_regular_model["Linear"] = mean_r2_2

In [52]:
linear_model2 = LinearRegression()
mean_ap, mean_r2, aps, all_predictions = backtest(stats2, linear_model2, years[5:], predictors2)
Model_error_MAP["Linear"] = mean_ap_1
Model_error_r2["Linear"] = mean_r2_2

In [53]:
#Prediction using Linear Regression:
over_65_games_2024 = stats_2024[stats_2024["G"] >= 65]
predictions_2024 = linear_model.predict(over_65_games_2024[predictors])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024.index)
combination_2024 = pd.concat([over_65_games_2024[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
314,Giannis Antetokounmpo,0.236619
398,Shai Gilgeous-Alexander,0.200812
242,Anthony Davis,0.197924
511,Victor Wembanyama,0.192774
143,Nikola Jokić,0.184716
340,Rudy Gobert,0.166736
124,Luka Dončić,0.161842
385,Chet Holmgren,0.148959
50,Nic Claxton,0.129818
254,LeBron James,0.128007


In [54]:
#Prediction using Linear Regression with engineered variables:
over_65_games_2024_2 = stats_2024_2[stats_2024_2["G"] >= 65]
predictions_2024 = linear_model2.predict(over_65_games_2024_2[predictors2])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024_2.index)
combination_2024 = pd.concat([over_65_games_2024_2[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
314,Giannis Antetokounmpo,0.236009
398,Shai Gilgeous-Alexander,0.202976
242,Anthony Davis,0.192176
143,Nikola Jokić,0.185019
511,Victor Wembanyama,0.181088
124,Luka Dončić,0.162683
340,Rudy Gobert,0.160319
385,Chet Holmgren,0.141084
254,LeBron James,0.129287
223,Tyrese Haliburton,0.126590


In [66]:
for model, error in Model_error_MAP_regular_model.items():
    print(f"For {model} the MAP is {error}")


For Ridge the MAP is 0.47337796919803465
For Random Forest the MAP is 0.5272513827337901
For XGBoost the MAP is 0.5737679498444628
For SVR the MAP is 0.1581575527040737
For Lasso the MAP is 0.47605468026998266
For Linear the MAP is 0.4768807352778601


In [67]:
for model, error in Model_error_r2_regular_model.items():
    print(f"For {model} the Mean r2 score is {error}")

For Ridge the Mean r2 score is 0.20632764973905682
For Random Forest the Mean r2 score is 0.38773525956396754
For XGBoost the Mean r2 score is 0.4245902608473405
For SVR the Mean r2 score is -1.0884712916883492
For Lasso the Mean r2 score is 0.20167066951025558
For Linear the Mean r2 score is 0.20165184281648585


In [68]:
for model, error in Model_error_MAP.items():
    print(f"For {model} the MAP score is {error}")

For Ridge the MAP score is 0.4693152099846445
For Random Forest the MAP score is 0.5103580176260846
For XGBoost the MAP score is 0.5634975367451635
For SVR the MAP score is 0.1614999540827289
For Lasso the MAP score is 0.4768807352778601
For Linear the MAP score is 0.4768807352778601


In [69]:
for model, error in Model_error_r2.items():
    print(f"For {model} the mean r2 score is {error}")

For Ridge the mean r2 score is 0.2058040822379013
For Random Forest the mean r2 score is 0.383827817458623
For XGBoost the mean r2 score is 0.4122562784866129
For SVR the mean r2 score is -1.0880312009587212
For Lasso the mean r2 score is 0.20165184281648585
For Linear the mean r2 score is 0.20165184281648585
